# Inputs
Get the files located here for example inputs to pyhf:

```
https://github.com/vladov3000/pyvshf/tree/master/hfout
```

# Input Setup

[1] Remove the first line of xml files
```
MonoJetOut/config/*.xml
```
You do not need to do this if you are using the data provided in the git repo.

In [3]:
!sed '14d' ../hfout/config/MyOneBinExample/Exclusion.xml > temp.xml 

But be careful, this command above is a hack and "irreversibly modifies" the xml config file.  If you run it twice, then it will crash everything when you try to run the xml2json command below. 

In [4]:
!cp temp.xml ../hfout/config/MyOneBinExample/Exclusion.xml

# pyhf Stuff

Start by importing the necessary tools

In [5]:
import logging
import json
import pyhf
from pyhf import Model,Workspace

Convert the xml workspace from Giuliano to a pyhf compatible JSON spec

In [12]:
!ls $(realpath ../hfout/config/MyOneBinExample/Exclusion.xml)

/Users/vlad/Documents/EPEprojects/pyvshf/hfout/config/MyOneBinExample/Exclusion.xml


In [13]:
!pyhf xml2json --hide-progress $(realpath ../hfout/config/MyOneBinExample/Exclusion.xml) --basedir $(realpath ../hfout) | tee monojet.json

{
    "channels": [
        {
            "name": "SR_cuts",
            "samples": [
                {
                    "data": [
                        1.1137869358062744
                    ],
                    "modifiers": [
                        {
                            "data": {
                                "hi": 0.820103,
                                "lo": 1.33253
                            },
                            "name": "KtScaleTop",
                            "type": "normsys"
                        },
                        {
                            "data": {
                                "hi": 1.26857,
                                "lo": 0.840855
                            },
                            "name": "JES",
                            "type": "normsys"
                        }
                    ],
                    "name": "Top"
                },
                {
                    "data": [
                        3

Validate that the spec you just created is valid

In [14]:
import json, requests, jsonschema
workspace = json.load(open('monojet.json'))

# This line breaks so I just got the schema from the url
# schema = requests.get('https://diana-hep.org/pyhf/schemas/1.0.0/workspace.json').json()
schema = {
    "$schema": "http://json-schema.org/draft-06/schema#",
    "$id": "https://scikit-hep.org/pyhf/schemas/1.0.0/workspace.json",
    "$ref": "defs.json#/definitions/workspace"
}

jsonschema.validate(instance=workspace, schema=schema)

Or by using the stuff from pyhf (because Giordon says its better in a magic way)

In [15]:
pyhf.utils.validate(workspace, 'workspace.json')

In [19]:
ws = pyhf.Workspace(workspace)

Now extract the model from the workspace

In [21]:
m = ws.model()

Print the "data" associated with the model.  Note that the "data" here has two components :
- The observed event counts in a given bin
- The NP's that will eventually be constrained.  These are referred to as "data" in the pyhf and HF lingo because they are indeed measurements that come from the data (e.g. luminosity, JES).

This is a bit of a subtle point and confusing but essential to allow us to get at the technical bits and pieces we are interested in examining.

In [22]:
d = ws.data(m)
print(d)

[3.0, 0.0, 0.0, 0.0, 1.0]


Display the initial values of the nuissance parameters in our model.

In [23]:
m.config.suggested_init()

[0.0, 0.0, 0.0, 1.0, 1.0]

Evaluate the value of the likelihood of the model and the data at the value of the NP's that are their initial starting values.

In [24]:
m.logpdf(m.config.suggested_init(), d)

array([-8.21114099])

You can toggle the available backends here, which is the machinery that will be used to perform the likelihood minimization.

This call will let you know what backends you have at your disposal, and then you can perform the minimization below by choosing from among the entries in this list.

In [25]:
pyhf.get_backend()

(<pyhf.tensor.numpy_backend.numpy_backend at 0x10daa8dd0>,
 <pyhf.optimize.opt_scipy.scipy_optimizer at 0x10daa8e50>)

We will choose the second one, which on Sam's machine is scipy_optimizer

Using this optimizer we will now perform the unconstrained fit, meaning that all NP's and the POI are free to be pulled.

This will return the values of the NP's that minimize the likelihood

In [32]:
pyhf.set_backend("numpy")
bfit = pyhf.infer.mle.fit(d, m)

In [31]:
# Try scipy backend, though it does not work for me
# opt = pyhf.get_backend()[1]
# bfit = opt.unconstrained_bestfit(pyhf.utils.loglambdav, d, m, m.config.suggested_init(), m.config.suggested_bounds() )

We can now print the values of those NP's.  Note that since the input data is the asimov dataset, they are very close to the input values

In [33]:
bfit

array([ 6.90103424e-02, -1.97967784e-01,  7.26203092e-08,  1.00002843e+00,
        2.03234500e-15])

And you can show the "actual" data which is the event counts that we are fitting out model to.

In [34]:
m.expected_actualdata(bfit)

array([4.46321158])

... or we can look at the full "data" list, which as we recall is both the event counts and the nuissance parameters which come into the constraint terms in the likelihood.

In [35]:
m.expected_data(bfit)

array([ 4.46321158e+00, -1.97967784e-01,  6.90103424e-02,  7.26203092e-08,
        1.00002843e+00])

Finally, we can take the difference between the fitted NPs and the initial values that we started with and that gives us the pull of that NP.

In [36]:
bfit-m.config.suggested_init()

array([ 6.90103424e-02, -1.97967784e-01,  7.26203092e-08,  2.84285566e-05,
       -1.00000000e+00])

# Open Point
The open, and important point, is that the manner in which the error on a given NP is not well defined here in pyhf it seems.